# Colab output wrapper

In [25]:
# wrap the output in colab cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip install tensorflow --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.12.0
    Uninstalling tensorflow-2.12.0:


# Install Transformers

In [3]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]

# Read input file from Google Drive

In [4]:
# open and read the file from google drive
file = open("/kaggle/input/annualr/training/training/annual_reports/102.txt", "r")
FileContent = file.read().strip()

In [ ]:
# display file content
FileContent 

In [6]:
# total characters in the file
len(FileContent) 

169640

In [7]:
import torch

# Load the Model and Tokenizer

In [10]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import tensorflow as tf
checkpoint = "randomshit11/fin-bert-1st-shit"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Some model statistics

In [11]:
# max tokens including the special tokens
tokenizer.model_max_length 

512

In [12]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence 

511

In [13]:
# number of special tokens
tokenizer.num_special_tokens_to_add() 

1

# Convert file content to sentences

In [14]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

Token indices sequence length is longer than the specified maximum sequence length for this model (762 > 512). Running this sequence through the model will result in indexing errors


762

# Create the chunks

In [16]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

67

# Some checks

In [17]:
[len(tokenizer.tokenize(c)) for c in chunks]

[503,
 762,
 447,
 487,
 510,
 498,
 501,
 490,
 503,
 493,
 504,
 498,
 451,
 477,
 478,
 489,
 466,
 497,
 502,
 496,
 384,
 481,
 500,
 492,
 506,
 489,
 476,
 495,
 463,
 465,
 496,
 475,
 486,
 506,
 479,
 495,
 505,
 414,
 246,
 496,
 483,
 502,
 434,
 455,
 488,
 256,
 491,
 226,
 508,
 508,
 490,
 485,
 451,
 367,
 463,
 427,
 429,
 470,
 480,
 430,
 507,
 492,
 430,
 195,
 464,
 466,
 203]

In [18]:
[len(tokenizer(c).input_ids) for c in chunks]

[504,
 763,
 448,
 488,
 511,
 499,
 502,
 491,
 504,
 494,
 505,
 499,
 452,
 478,
 479,
 490,
 467,
 498,
 503,
 497,
 385,
 482,
 501,
 493,
 507,
 490,
 477,
 496,
 464,
 466,
 497,
 476,
 487,
 507,
 480,
 496,
 506,
 415,
 247,
 497,
 484,
 503,
 435,
 456,
 489,
 257,
 492,
 227,
 509,
 509,
 491,
 486,
 452,
 368,
 464,
 428,
 430,
 471,
 481,
 431,
 508,
 493,
 431,
 196,
 465,
 467,
 204]

## With special tokens added

In [19]:
sum([len(tokenizer(c).input_ids) for c in chunks])

31068

In [20]:
len(tokenizer(FileContent).input_ids)

31429

## Without special tokens added

In [21]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

31001

In [22]:
len(tokenizer.tokenize(FileContent))

31428

# Get the inputs

In [23]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

# Output

In [24]:
for i, input in enumerate(inputs):
    try:
        output = model.generate(**input)
        generated_text = tokenizer.decode(*output, skip_special_tokens=True)
        print(generated_text)
    except Exception as e:
        print(f" ")


: Public Company Annual Report & Accounts 2005 Connecting...to the world’s no.1 online Casino and Poker room THE WORLD’S NO.1 ONLINE members AND POKERs 888 is one of the world’s most popular online gaming entertainment companies.Established in 1997 by pioneers of the online gaming industry,
 
: 888’s OPERATIONAl ARm IN GIb RAl TAR IS RESPONSIbl E f Or Th E d Ay-To-d Ay Ru NNING Of Th E GROu P’S fl AGSh IP GAm ING b RANd S ANd PROduc TS
: Of The World: 8% 1 8% 1 3 4 The Worlds No.1 oNli Casi No aNd Poker r oom We have over 23 millio N egis TerouNTs World WiPaes Couries TerouNTs World WiPaes Casi No-
: Localising offer both country and consumer segment is a key component driving both trial and repeat usage of our products.This helps to maintain high awareness and ensure flexibility in targeted local communication whilst guaranteeing global consistency in messages.Integrated marketing campaigns: a consistent communication approach with tailored activities leveraged through all channels.Fo